In [2]:


val job_skills_df = Seq(
    (1, "Data Engineer", "SQL"),
    (2, null, "Python"),
    (3, null, "AWS"),
    (4, null, "Snowflake"),
    (5, null, "Apache Spark"),
    (6, "Web Developer", "Java"),
    (7, null, "HTML"),
    (8, null, "CSS"),
    (9, "Data Scientist", "Python"),
    (10, null, "Machine Learning"),
    (11, null, "Deep Learning"),
    (12, null, "Tableau")
).toDF("row_id", "job_role", "skills")


job_skills_df.show(false)


+------+--------------+----------------+
|row_id|job_role      |skills          |
+------+--------------+----------------+
|1     |Data Engineer |SQL             |
|2     |null          |Python          |
|3     |null          |AWS             |
|4     |null          |Snowflake       |
|5     |null          |Apache Spark    |
|6     |Web Developer |Java            |
|7     |null          |HTML            |
|8     |null          |CSS             |
|9     |Data Scientist|Python          |
|10    |null          |Machine Learning|
|11    |null          |Deep Learning   |
|12    |null          |Tableau         |
+------+--------------+----------------+



job_skills_df: org.apache.spark.sql.DataFrame = [row_id: int, job_role: string ... 1 more field]


In [3]:
val flag_df = job_skills_df.withColumn("flag", when($"job_role".isNotNull, lit(1)).otherwise(lit(0)))
flag_df.show(false)

+------+--------------+----------------+----+
|row_id|job_role      |skills          |flag|
+------+--------------+----------------+----+
|1     |Data Engineer |SQL             |1   |
|2     |null          |Python          |0   |
|3     |null          |AWS             |0   |
|4     |null          |Snowflake       |0   |
|5     |null          |Apache Spark    |0   |
|6     |Web Developer |Java            |1   |
|7     |null          |HTML            |0   |
|8     |null          |CSS             |0   |
|9     |Data Scientist|Python          |1   |
|10    |null          |Machine Learning|0   |
|11    |null          |Deep Learning   |0   |
|12    |null          |Tableau         |0   |
+------+--------------+----------------+----+



flag_df: org.apache.spark.sql.DataFrame = [row_id: int, job_role: string ... 2 more fields]


In [5]:
import org.apache.spark.sql.expressions._

val running_sum_df = flag_df.withColumn("running_sum", sum($"flag").over(Window.orderBy($"row_id"))).drop("flag")

running_sum_df.show(false)

+------+--------------+----------------+-----------+
|row_id|job_role      |skills          |running_sum|
+------+--------------+----------------+-----------+
|1     |Data Engineer |SQL             |1          |
|2     |null          |Python          |1          |
|3     |null          |AWS             |1          |
|4     |null          |Snowflake       |1          |
|5     |null          |Apache Spark    |1          |
|6     |Web Developer |Java            |2          |
|7     |null          |HTML            |2          |
|8     |null          |CSS             |2          |
|9     |Data Scientist|Python          |3          |
|10    |null          |Machine Learning|3          |
|11    |null          |Deep Learning   |3          |
|12    |null          |Tableau         |3          |
+------+--------------+----------------+-----------+



import org.apache.spark.sql.expressions._
running_sum_df: org.apache.spark.sql.DataFrame = [row_id: int, job_role: string ... 2 more fields]


In [12]:
running_sum_df.withColumn("job_role", first($"job_role").over(Window.partitionBy($"running_sum"))
                         ).drop("running_sum").show(false)

+------+--------------+----------------+
|row_id|job_role      |skills          |
+------+--------------+----------------+
|1     |Data Engineer |SQL             |
|2     |Data Engineer |Python          |
|3     |Data Engineer |AWS             |
|4     |Data Engineer |Snowflake       |
|5     |Data Engineer |Apache Spark    |
|6     |Web Developer |Java            |
|7     |Web Developer |HTML            |
|8     |Web Developer |CSS             |
|9     |Data Scientist|Python          |
|10    |Data Scientist|Machine Learning|
|11    |Data Scientist|Deep Learning   |
|12    |Data Scientist|Tableau         |
+------+--------------+----------------+

